In [ ]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import path
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import seaborn as sns


import os
print(os.listdir("../input"))

# read data in pandas dataframe
# df_train =  pd.read_csv('../input/train.csv', nrows = 150000, parse_dates=["pickup_datetime"])
df_train =  pd.read_csv('../input/train.csv', nrows = 50000, parse_dates=["pickup_datetime"])
print(len(df_train))
print(df_train.dtypes)
df_train.head()

**Data Cleaning**

In [ ]:
# check and drop the missing values
print(df_train.isnull().sum())
df_train = df_train.dropna(how = 'any', axis = 'rows')

In [ ]:
# check for outliers in fare_amount 
df_train["fare_amount"].hist(bins=100)
plt.xlabel('fare $USD')
plt.ylabel('frequency')
plt.title('fare_amount distribution')

print(df_train["fare_amount"].describe())

# the minimal fare_amount is negative, which is not realistic, so drop them
df_train = df_train[df_train.fare_amount>=0]

In [ ]:
# check for outliers in passenger_count 
df_train["passenger_count"].hist(bins=100, figsize=(14,3))
plt.xlabel('fare $USD')
plt.ylabel('frequency')
plt.title('passenger_count distribution')

print(df_train["passenger_count"].describe())

# passenger_count of zero or larger than 6 (a SUV cab) seems to be not realistic, so drop them
df_train = df_train[(df_train['passenger_count']>0) & (df_train['passenger_count']<=6)]

In [ ]:
# Googling latitudes and longitudes give such information
# Latitudes range from -90 to 90.
# Longitudes range from -180 to 180.
print(df_train['pickup_latitude'].describe())
print(df_train['pickup_longitude'].describe())
print(df_train['dropoff_latitude'].describe())
print(df_train['dropoff_longitude'].describe())

# it shows some data errors in latitude and longitude,
# including outliers and wrong order when inserted latitude and longitude into dataset
# by using Google map, the latitude and longitude coordinate of New York City is:
nyc = (40.730610, -73.935242)

# so drop trips that are out of New York City and its nearby areas
def city_nearby_areas(df, city):
    return (df['pickup_longitude'] >= city[1]-1.5) & (df['pickup_longitude'] <= city[1]+1.5) & \
           (df['pickup_latitude'] >= city[0]-1.5) & (df['pickup_latitude'] <= city[0]+1.5) & \
           (df['dropoff_longitude'] >= city[1]-1.5) & (df['dropoff_longitude'] <= city[1]+1.5) & \
           (df['dropoff_latitude'] >= city[0]-1.5) & (df['dropoff_latitude'] <= city[0]+1.5)

df_train = df_train[city_nearby_areas(df_train, nyc)]
print(len(df_train))

In [ ]:
# a function to calculate distance between two latitude longitude points.
# this function is based on https://stackoverflow.com/questions/27928/
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295 # Pi/180
    a = 0.5 - np.cos((lat2 - lat1) * p)/2 + np.cos(lat1 * p) * np.cos(lat2 * p) * (1 - np.cos((lon2 - lon1) * p)) / 2
    return 12742 * np.arcsin(np.sqrt(a)) # 2*R*asin...

# calculate the distance in kilometers between pickup and dropoff postion of each trip 
df_train['distance_km'] = df_train.apply(lambda x: distance(x["pickup_latitude"], x["pickup_longitude"], \
                                   x["dropoff_latitude"], x["dropoff_longitude"]), axis=1)

In [ ]:
# check for outliers in distance_km 
print(df_train["distance_km"].describe())

plt.scatter(df_train["distance_km"], df_train["fare_amount"], alpha=0.2)
plt.xlabel('distance in kilometers')
plt.ylabel('fare $USD')
plt.title('distance vs. fare')

# the plot shows there are trips with zero distance but with a non-zero fare
# drop trips with zero distance
df_train = df_train[df_train["distance_km"] >= 0.1]
print(len(df_train))

# the plot also shows long trips but with low fare, just leave them now and I will analyze them later
# df_train[df_train.fare_amount > 50]

In [ ]:
plt.scatter(df_train["pickup_longitude"], df_train["pickup_latitude"], alpha=0.2)
plt.xlim(nyc[1]-0.1, nyc[1])
plt.ylim(nyc[0]-0.05, nyc[0]+0.05)
plt.xlabel('pickup_longitude')
plt.ylabel('pickup_latitude')
plt.title('pickup positions')

# df_train = df_train[(df_train["distance_km"] < 80)]

In [ ]:
# Airport trip to/from JFK and any location in Manhattan is a flat fare
# Airport trip from JFK to other New York City destinations is a metered fare
# Airport trip to Newark Liberty International Airport is a metered fare with a Newark Surchargeand 
# Airport trip to LaGuardia Airport is a metered fare

# find if a point is in the Manhattan area
# roughly divide Manhattan to three parts to simplify calculation
Manhattan_A = [(40.706476, -74.025303), (40.700178, -74.005985), (40.714462, -73.975092), (40.725429, -74.023245)]
Manhattan_B = [(40.725429, -74.023245), (40.714462, -73.975092), (40.796551, -73.914160), (40.819727, -73.964973)]
Manhattan_C = [(40.819727, -73.964973), (40.804786, -73.930812), (40.834324, -73.934135), (40.839926, -73.949915)]
Manhattan_D = [(40.839926, -73.949915), (40.834324, -73.934135), (40.872095, -73.909464), (40.879273, -73.930286)]

def in_area(area, place):
    p = path.Path(area)
    res = p.contains_point(place)
    return res

def in_Manhattan(place):
    res = in_area(Manhattan_A, place) or in_area(Manhattan_B, place) \
          or in_area(Manhattan_C, place) or in_area(Manhattan_D, place) 
    return res

in_Manhattan([40.692343, -73.999933])

# Find if the dropoff place is an airport, and using one-hot encoding to represent this.

# jfk = (40.6413, -73.7781)
# ewr = (40.6895, -74.1745)

# def to_airport(lat1, lon1):
#     if distance(lat1, lon1, jfk[0], jfk[1]) <=3:
#         return "airport_jfk"
#     elif distance(lat1, lon1, ewr[0], ewr[1]) <=3:
#         return "airport_ewr"
#     elif distance(lat1, lon1, lgr[1], lgr[0]) <=3:
#         return "airport_lgr"
#     return "not_airport"

# df_train["airport"] = df_train.apply(lambda x: to_airport(x["dropoff_latitude"], x["dropoff_longitude"]), axis=1)


In [ ]:
# def airport_fare(airport):
#     df_airport = df_train[df_train["airport"] == airport]
#     airport_top_fare = df_airport.groupby('fare_amount')['fare_amount'].count() \
#                                                       .reset_index(name='count') \
#                                                       .sort_values(['count'], ascending=False) \
#                                                       .head(5)
#     weighted_average_fare = (airport_top_fare["fare_amount"] * airport_top_fare["count"]).sum() / airport_top_fare["count"].sum()
#     print(airport_top_fare)
#     return weighted_average_fare

# jfk_fare = airport_fare("airport_jfk").round(decimals = 2)
# ewr_fare = airport_fare("airport_ewr").round(decimals = 2)
# lgr_fare = airport_fare("airport_lgr").round(decimals = 2)
# print(jfk_fare, ewr_fare, lgr_fare)

In [ ]:
# df_airport = pd.get_dummies(df_train["airport"])
# df_train = df_train.join(df_airport)

**Extract Hour and Day From pickup_datetime**

Fare at night is different from day time.

The amount of traffic depends on the hour of the day, and it determines the duration of the trip and thus the fare. 

In [ ]:
# df_train['hour'] = df_train["pickup_datetime"].apply(lambda t: t.hour)
# df_train['year'] = df_train["pickup_datetime"].apply(lambda t: t.year-2009)
# df_train["day_of_week"] = df_train["pickup_datetime"].apply(lambda t: t.weekday())


def hour_type(hour):
    if hour in list(range(20, 25)) + list(range(0,7)):
        return "night"
    elif hour in range(7, 10):
        return "morning_peak"
    elif hour in range(16, 20):
        return "afternoon_peak"
    else:
        return "normal_hour"

def extract_time(record):
    record['hour'] = record["pickup_datetime"].hour
    record['year'] = record["pickup_datetime"].year
    record["day_of_week"] = record["pickup_datetime"].weekday()
    record["weekday"] = 1 if record["day_of_week"] in range(0, 5) else 0
    record["hour_type"] = hour_type(record["hour"])
    return record

df_train = df_train.apply(lambda r: extract_time(r), axis=1)

df_hour_type = pd.get_dummies(df_train["hour_type"])
df_train = df_train.join(df_hour_type)

# Another method to find the day of week 
# day_of_week = pd.to_datetime(df_train["pickup_datetime"]).dt.weekday_name
# or
# day_of_week = pd.to_datetime(df_train["pickup_datetime"]).dt.dayofweek

There are several kinds of surcharge to taxi fare:

1. \$0.5 of additional night surcharge between 8PM - 6AM.
2. \$1 of peak hour weekday surcharge of Monday-Friday between 4PM-8PM.

In [ ]:
def remove_surage(record):
    fare_amount_no_surage = record["fare_amount"]
    if record["hour_type"] == "night":
        fare_amount_no_surage = fare_amount_no_surage - 0.5
    if record["weekday"] == 1 and record["hour_type"] == "afternoon_peak":
        fare_amount_no_surage = fare_amount_no_surage - 1
    return fare_amount_no_surage

df_train["fare_amount_no_surage"] = df_train.apply(lambda r: remove_surage(r), axis=1)


In [ ]:
df_train = df_train[df_train.fare_amount>=0]
df_train.head(8)

In [ ]:
# for year in list(df_train["year"].unique()):
#     temp_df = df_train[(df_train["airport"] == "not_airport") & (df_train["year"] == year)]
#     sns.lmplot(x="distance_km", y="fare_amount_no_surage", data=temp_df, fit_reg=True)
#     plt.title("simple regression for year = " + str(year))


In [ ]:
# a test to research the effect of feature "year" on fare to airport
airport_fare_byYear = pd.DataFrame(df_train.groupby(['airport', 'year'])['fare_amount'].mean())
airport_fare_byYear.reset_index(inplace=True)  
airport_fare_byYear

**Model**

linear regression model: 
    
    fare_amount ~ distance_km + passenger_count + year + weekday + afternoon_peak + morning_peak + night + normal_hour

In [ ]:
# X = df_train.drop('fare_amount_no_surage', axis=1)
# y = df_train['fare_amount_no_surage']

df_train_no_airport = df_train[df_train["airport"] == "not_airport"]
features = ["distance_km", "passenger_count", "weekday", "afternoon_peak", "morning_peak", "night", "normal_hour"]
year_coef_dic = {}

for year in list(df_train["year"].unique()):
    temp_df = df_train_no_airport[df_train_no_airport["year"] == year]
    temp_lm = LinearRegression()
    temp_lm.fit(temp_df[features], temp_df["fare_amount_no_surage"])
    year_coef_dic[year] = [temp_lm.intercept_] + list(temp_lm.coef_)
    
year_coef_dic

# X = df_train_no_airport.drop('fare_amount_no_surage', axis=1)
# y = df_train_no_airport['fare_amount_no_surage']

# X_train_df, X_test_df, y_train_df, y_test_df = train_test_split(X, y, test_size=0.25)
# X.shape, y.shape


In [ ]:
def adjust_fare(record, prediction):
    if record["night"] == 1:
        prediction = prediction + 0.5
    if record["weekday"] == 1 and record["afternoon_peak"] == 1:
        prediction = prediction + 1
    
    if record["airport"] == "airport_jfk":
        prediction = jfk_fare
    elif record["airport"] == "airport_ewr":
        prediction = ewr_fare
    elif record["airport"] == "airport_lgr":
        prediction = lgr_fare
    
    return prediction


In [ ]:
def calculate_pred(row):
    input_value = [1] + row[features].values.tolist()
    coef = year_coef_dic[row["year"]]
    y_pred = sum([a*b for a,b in zip(input_value,coef)]).round(decimals = 2)
    y_adjust_pred = adjust_fare(row, y_pred)
    return y_adjust_pred

df_train["prediction"] = df_train.apply(lambda row: calculate_pred(row), axis=1)
df_train.head(5)

In [ ]:
def plot_prediction_analysis(y, y_pred):    
    plt.scatter(y, y_pred)
    plt.xlabel("real y")
    plt.ylabel("predicted y")
    plt.plot([0, max(df_train["fare_amount"])], [0, max(df_train["fare_amount"])], color='red', linestyle='-', linewidth=0.5)
    rmse = np.sqrt(mean_squared_error(y, y_pred))
    plt.title('rmse = {:.2f}, with zero residual diagonal line'.format(rmse))
    plt.show()


In [ ]:
# prediction plot for training dataset
plot_prediction_analysis(df_train["fare_amount"], df_train["prediction"])

# df_predict_y.index = df_train.index
# df_train_predict = pd.concat([df_train, df_predict_y], axis=1)
# adjust_predict_y = df_train_predict.apply(lambda r: adjust_fare(r), axis=1)
# # adjust_predict_y.head()
# plot_prediction_analysis(df_train["fare_amount"], adjust_predict_y)

**Validation Dataset Experiment**

In [ ]:
df_validation =  pd.read_csv('../input/train.csv', skiprows = 200000, nrows = 50000, \
                             names = ["key","fare_amount","pickup_datetime","pickup_longitude","pickup_latitude","dropoff_longitude","dropoff_latitude","passenger_count"], \
                             parse_dates=["pickup_datetime"])
print(len(df_validation))
print(df_validation.dtypes)
print(df_validation.isnull().sum())
df_validation.head()

In [ ]:
# preprocessing dataset to extract features
def preprocess_dataset(df):
    df['distance_km'] = df.apply(lambda x: distance(x["pickup_latitude"], x["pickup_longitude"], \
                                   x["dropoff_latitude"], x["dropoff_longitude"]), axis=1)
    df["airport"] = df.apply(lambda x: to_airport(x["dropoff_latitude"], x["dropoff_longitude"]), axis=1)
    df = df.apply(lambda r: extract_time(r), axis=1)
    df_hour_type = pd.get_dummies(df["hour_type"])
    df = df.join(df_hour_type)
    df["fare_amount_no_surage"] = df.apply(lambda r: remove_surage(r), axis=1)
    return df

In [ ]:
df_validation = preprocess_dataset(df_validation)
df_validation["prediction"] = df_validation.apply(lambda row: calculate_pred(row), axis=1)

# prediction plot for testing dataset
plot_prediction_analysis(df_validation["fare_amount"], df_validation["prediction"])

In [ ]:
print(df_validation.isnull().sum())

**Fit Test Dataset to Model**

In [ ]:
df_test = pd.read_csv('../input/test.csv', parse_dates=["pickup_datetime"])

In [ ]:
df_test.head()

In [ ]:
df_test['distance_km'] = df_test.apply(lambda x: distance(x["pickup_latitude"], x["pickup_longitude"], \
                                   x["dropoff_latitude"], x["dropoff_longitude"]), axis=1)

df_test["airport"] = df_test.apply(lambda x: to_airport(x["dropoff_latitude"], x["dropoff_longitude"]), axis=1)
df_airport = pd.get_dummies(df_test["airport"])
df_test = df_test.join(df_airport)

df_test['hour'] = df_test["pickup_datetime"].apply(lambda t: t.hour)
df_test['year'] = df_test["pickup_datetime"].apply(lambda t: t.year)
df_test["weekdays"] = df_test["pickup_datetime"].apply(lambda t: 1 if t.weekday() in range(0,5) else 0)

In [ ]:
filename = './output/baseline_linear'

test_X = df_test[features].values
y_pred_final = fare_lm.predict(test_X).round(decimals = 2)

# print(y_pred_final.shape)
# print(y_pred_final[:10])

submission = pd.DataFrame(
    {'key': df_test.key, 'fare_amount': y_pred_final},
    columns = ['key', 'fare_amount'])
submission.to_csv('submission.csv', index = False)